In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fashion-campus/click_stream.csv
/kaggle/input/fashion-campus/product.csv
/kaggle/input/fashion-campus/customer.csv
/kaggle/input/fashion-campus/transactions.csv


In [2]:
pd.set_option('display.max_columns', 10000)

# Read data

In [3]:
click_stream = pd.read_csv('../input/fashion-campus/click_stream.csv')
customers = pd.read_csv('../input/fashion-campus/customer.csv')
products = pd.read_csv('../input/fashion-campus/product.csv', on_bad_lines='skip')
transactions = pd.read_csv('../input/fashion-campus/transactions.csv')

# Labeling

In [4]:
#Let's filter data who mades success transaction
transactions_success = transactions[transactions['payment_status'] == 'Success']

#Remove qoutes in in product_metadata
import ast

transactions_success['product_metadata'] = transactions_success['product_metadata'].apply(ast.literal_eval)
transactions_success[['product_metadata']]

#Remove list in each row in product_metadata
transactions_success = transactions_success.explode('product_metadata')

#Crack and create new columns
transactions_success1 = pd.concat([transactions_success.drop(['product_metadata'], axis=1), transactions_success['product_metadata'].apply(pd.Series)], axis=1)
transactions_success1

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,created_at,customer_id,booking_id,session_id,payment_method,payment_status,promo_amount,promo_code,shipment_fee,shipment_date_limit,shipment_location_lat,shipment_location_long,total_amount,product_id,quantity,item_price
0,2018-07-29T15:22:01.458193Z,5868,186e2bee-0637-4710-8981-50c2d737bc42,3abaa6ce-e320-4e51-9469-d9f3fa328e86,Debit Card,Success,1415,WEEKENDSERU,10000,2018-08-03T05:07:24.812676Z,-8.227893,111.969107,199832,54728,1,191247
1,2018-07-30T12:40:22.365620Z,4774,caadb57b-e808-4f94-9e96-8a7d4c9898db,2ee5ead1-f13e-4759-92df-7ff48475e970,Credit Card,Success,0,NaN,10000,2018-08-03T01:29:03.415705Z,3.013470,107.802514,155526,16193,1,145526
2,2018-09-15T11:51:17.365620Z,4774,6000fffb-9c1a-4f4a-9296-bc8f6b622b50,93325fb6-eb00-4268-bb0e-6471795a0ad0,OVO,Success,0,NaN,10000,2018-09-18T08:41:49.422380Z,-2.579428,115.743885,550696,53686,4,135174
3,2018-11-01T11:23:48.365620Z,4774,f5e530a7-4350-4cd1-a3bc-525b5037bcab,bcad5a61-1b67-448d-8ff4-781d67bc56e4,Credit Card,Success,0,NaN,0,2018-11-05T17:42:27.954235Z,-3.602334,120.363824,271012,20228,1,271012
4,2018-12-18T11:20:30.365620Z,4774,0efc0594-dbbf-4f9a-b0b0-a488cfddf8a2,df1042ab-13e6-4072-b9d2-64a81974c51a,Credit Card,Success,0,NaN,0,2018-12-23T17:24:07.361785Z,-3.602334,120.363824,198753,55220,1,198753
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
852578,2020-11-12T18:19:20.074873Z,25121,7fdc05d1-234b-4889-a343-587ffc0635d5,33c3bf4f-0f24-4eb8-879d-906bf7e7736a,Debit Card,Success,0,NaN,10000,2020-11-18T18:06:06.493534Z,-6.286732,106.902229,287688,42915,1,277688
852580,2021-12-09T07:35:18.222046Z,34829,ccde6b83-ee42-4cce-b2cd-976574886eac,954e1211-4d45-4d0b-ba84-53e80119592c,OVO,Success,5010,AZ2022,10000,2021-12-12T07:52:44.803822Z,-8.677264,115.176947,359203,19394,1,354213
852581,2021-12-11T01:20:36.944593Z,99645,5bcd3434-1dc6-44bb-ae63-fd19714cbd6c,e6e6c108-abd0-49a2-b0a5-691117f9cfe1,Credit Card,Success,2181,WEEKENDSERU,5000,2021-12-12T13:28:22.848703Z,-6.185498,106.831093,1158159,13079,4,288835
852582,2019-12-18T11:01:09.623382Z,87319,af7f291a-b01b-4bfd-89a9-237b16368b15,bb6e5426-a3f3-426a-b99d-79173eaa234b,Credit Card,Success,4503,AZ2022,0,2019-12-23T22:56:46.831720Z,-7.155977,110.891858,184143,11312,1,188646


In [5]:
#Let's merge table to picture better understanding
customers_trans = customers.merge(transactions_success1, on = 'customer_id', how = 'left')

#Convert object type in created_at columns to datetime with format year-month
customers_trans['created_at'] = pd.to_datetime(customers_trans['created_at'])
customers_trans['created_at'] = customers_trans['created_at'].dt.strftime('%Y-%m')

# There 49758 customers does not have transaction, so I ignore them.
customers_trans = customers_trans[~customers_trans['booking_id'].isnull()]

#There customers do not use voucher, let's create new columns 
customers_trans['use_promo'] = customers_trans['promo_amount'].apply(lambda x: 1 if x >= 1 else 0)

#Display the table
customers_trans

,customer_id,first_name,last_name,username,email,gender,birthdate,device_type,device_id,device_version,...,promo_code,shipment_fee,shipment_date_limit,shipment_location_lat,shipment_location_long,total_amount,product_id,quantity,item_price,use_promo
2,7279,Bakiman,Simanjuntak,3250e5a3-1d23-4675-a647-3281879d42be,3250e5a3_1d23_4675_a647_3281879d42be@startupca...,M,1989-01-23,iOS,d13dde0a-6ae1-43c3-83a7-11bbb922730b,iPad; CPU iPad OS 4_2_1 like Mac OS X,...,WEEKENDSERU,15000.0,2020-08-26T05:14:04.908999Z,0.621070,111.101844,844751.0,18015.0,1.0,305982.0,1
3,7279,Bakiman,Simanjuntak,3250e5a3-1d23-4675-a647-3281879d42be,3250e5a3_1d23_4675_a647_3281879d42be@startupca...,M,1989-01-23,iOS,d13dde0a-6ae1-43c3-83a7-11bbb922730b,iPad; CPU iPad OS 4_2_1 like Mac OS X,...,WEEKENDSERU,15000.0,2020-08-26T05:14:04.908999Z,0.621070,111.101844,844751.0,2401.0,2.0,263311.0,1
4,7279,Bakiman,Simanjuntak,3250e5a3-1d23-4675-a647-3281879d42be,3250e5a3_1d23_4675_a647_3281879d42be@startupca...,M,1989-01-23,iOS,d13dde0a-6ae1-43c3-83a7-11bbb922730b,iPad; CPU iPad OS 4_2_1 like Mac OS X,...,NaN,10000.0,2020-09-04T05:49:01.977704Z,-6.281795,106.890417,391341.0,12449.0,1.0,381341.0,0
5,7279,Bakiman,Simanjuntak,3250e5a3-1d23-4675-a647-3281879d42be,3250e5a3_1d23_4675_a647_3281879d42be@startupca...,M,1989-01-23,iOS,d13dde0a-6ae1-43c3-83a7-11bbb922730b,iPad; CPU iPad OS 4_2_1 like Mac OS X,...,NaN,0.0,2020-09-14T11:26:20.142928Z,-8.631607,116.428436,154220.0,25820.0,1.0,154220.0,0
6,7279,Bakiman,Simanjuntak,3250e5a3-1d23-4675-a647-3281879d42be,3250e5a3_1d23_4675_a647_3281879d42be@startupca...,M,1989-01-23,iOS,d13dde0a-6ae1-43c3-83a7-11bbb922730b,iPad; CPU iPad OS 4_2_1 like Mac OS X,...,NaN,10000.0,2020-09-19T22:38:44.292440Z,0.825219,123.396780,412502.0,25157.0,1.0,247557.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1250788,40573,Yani,Usada,f5a2ab29-481d-49e6-960e-cb864ee1ed5a,f5a2ab29_481d_49e6_960e_cb864ee1ed5a@startupca...,F,1987-09-10,Android,b6bc29e0-5a22-47af-b31e-0972aafd4cab,Android 3.2.6,...,NaN,10000.0,2021-10-24T08:18:20.393551Z,-7.179850,110.039105,311938.0,48507.0,2.0,150969.0,0
1250789,40573,Yani,Usada,f5a2ab29-481d-49e6-960e-cb864ee1ed5a,f5a2ab29_481d_49e6_960e_cb864ee1ed5a@startupca...,F,1987-09-10,Android,b6bc29e0-5a22-47af-b31e-0972aafd4cab,Android 3.2.6,...,WEEKENDSERU,10000.0,2022-01-26T01:41:11.712735Z,-6.286999,106.838482,110483.0,42935.0,1.0,105015.0,1
1250790,40573,Yani,Usada,f5a2ab29-481d-49e6-960e-cb864ee1ed5a,f5a2ab29_481d_49e6_960e_cb864ee1ed5a@startupca...,F,1987-09-10,Android,b6bc29e0-5a22-47af-b31e-0972aafd4cab,Android 3.2.6,...,NaN,25000.0,2022-03-14T02:16:33.321801Z,0.583145,113.530523,276290.0,54808.0,1.0,251290.0,0
1250791,40573,Yani,Usada,f5a2ab29-481d-49e6-960e-cb864ee1ed5a,f5a2ab29_481d_49e6_960e_cb864ee1ed5a@startupca...,F,1987-09-10,Android,b6bc29e0-5a22-47af-b31e-0972aafd4cab,Android 3.2.6,...,NaN,10000.0,2022-04-29T06:40:10.012855Z,-7.819493,110.699877,258774.0,7158.0,1.0,248774.0,0


In [6]:

#Let's grouping table based on customer_id and create_att and find out total transaction customer made each month
customers_trans1 = customers_trans.groupby(['customer_id', 'created_at', 'gender', 'device_type', 'birthdate'], as_index = True).agg({'booking_id':'count', 'use_promo':'sum', 'quantity':'sum', 'total_amount':'sum'}).reset_index()

#Convert object type in created_at columns to datetime, again.
customers_trans1['created_at'] = pd.to_datetime(customers_trans1['created_at'])

#Rename column name booking_id to total_transaction for better understanding
customers_trans1.rename(columns = {'booking_id':'total_transaction', 'quantity':'quantity_item'}, inplace = True)
customers_trans1.sort_values(['customer_id', 'created_at'], inplace = True)
customers_trans1

,customer_id,created_at,gender,device_type,birthdate,total_transaction,use_promo,quantity_item,total_amount
0,3,2018-08-01,M,Android,1986-07-12,1,1,5.0,826597.0
1,3,2018-09-01,M,Android,1986-07-12,1,0,1.0,163784.0
2,3,2018-10-01,M,Android,1986-07-12,1,1,1.0,124932.0
3,3,2018-11-01,M,Android,1986-07-12,1,1,1.0,247902.0
4,3,2018-12-01,M,Android,1986-07-12,1,1,1.0,181237.0
...,...,...,...,...,...,...,...,...,...
565536,99998,2020-07-01,F,Android,2014-05-21,2,0,2.0,1068674.0
565537,99998,2021-01-01,F,Android,2014-05-21,1,1,1.0,366178.0
565538,99998,2021-07-01,F,Android,2014-05-21,1,0,2.0,485330.0
565539,99998,2022-01-01,F,Android,2014-05-21,16,16,22.0,67558976.0


In [7]:
#Labeling Process Churn: If row and row+1 has same customer_id so subtract row i+1 - row and so on divide with timedelta to remove format default timedelta in created_at column rows, if otherwise subtract row i - row i and so on.
a = []
for i in range(customers_trans1.shape[0] - 1) :
    if customers_trans1.loc[i, 'customer_id'] == customers_trans1.loc[i+1, 'customer_id'] :
        a.append((customers_trans1.loc[i+1,'created_at'] - customers_trans1.loc[i,'created_at'])/np.timedelta64(1,'M'))
    else :
        a.append((customers_trans1.loc[i, 'created_at'] - customers_trans1.loc[i,'created_at'])/np.timedelta64(1,'M'))

In [8]:
#Let's a's list transformation to dataframe
diff = pd.DataFrame(a, columns = ["diff"])
diff = round(diff)

In [9]:
#Let's unite table between customers_trans1 and diff
customers_trans1 = pd.concat([customers_trans1, diff], axis=1)
customers_trans1

,customer_id,created_at,gender,device_type,birthdate,total_transaction,use_promo,quantity_item,total_amount,diff
0,3,2018-08-01,M,Android,1986-07-12,1,1,5.0,826597.0,1.0
1,3,2018-09-01,M,Android,1986-07-12,1,0,1.0,163784.0,1.0
2,3,2018-10-01,M,Android,1986-07-12,1,1,1.0,124932.0,1.0
3,3,2018-11-01,M,Android,1986-07-12,1,1,1.0,247902.0,1.0
4,3,2018-12-01,M,Android,1986-07-12,1,1,1.0,181237.0,2.0
...,...,...,...,...,...,...,...,...,...,...
565536,99998,2020-07-01,F,Android,2014-05-21,2,0,2.0,1068674.0,6.0
565537,99998,2021-01-01,F,Android,2014-05-21,1,1,1.0,366178.0,6.0
565538,99998,2021-07-01,F,Android,2014-05-21,1,0,2.0,485330.0,6.0
565539,99998,2022-01-01,F,Android,2014-05-21,16,16,22.0,67558976.0,0.0


In [10]:
#There a customer in last row who made once transaction let's fill value with 0
customers_trans1['diff'].fillna(0, inplace = True)

In [11]:
#Churn method. Customer nonchurn: 0 and churn: 1
def labelling_churn(df) : 
    if df["diff"] == 1 : 
        return 0
    else :
        return 1

In [12]:
#Apply method labelling_churn
customers_trans1["label_churn"] = customers_trans1.apply(labelling_churn, axis = 1) 

In [13]:
customers_churn = customers_trans1[['customer_id', 'label_churn']]
customers_churn

,customer_id,label_churn
0,3,0
1,3,0
2,3,0
3,3,0
4,3,1
...,...,...
565536,99998,1
565537,99998,1
565538,99998,1
565539,99998,1


In [14]:
customers_churn['label_churn'].value_counts(normalize = True)

0    0.565017
1    0.434983
Name: label_churn, dtype: float64

# Feature Engineering and Feature Selection

## 1. total_transaction columns

In [112]:
customers_trans1.columns

Index(['customer_id', 'created_at', 'gender', 'device_type', 'birthdate',
       'total_transaction', 'use_promo', 'quantity_item', 'total_amount',
       'diff', 'label_churn'],
      dtype='object')

In [113]:
asess_1 = pd.crosstab(customers_trans1["gender"], customers_trans1["label_churn"], margins=True, margins_name="Total")
asess_1

label_churn,0,1,Total
gender,,,
F,202832,156995,359827
M,116708,89006,205714
Total,319540,246001,565541


In [114]:
def nonevent(df) : 
    return (df[0]/319540)
def event(df) : 
    return (df[1]/246001)

    
asess_1["%nonevent"] = asess_1.apply(nonevent, axis = 1)
asess_1["%event"] = asess_1.apply(event, axis = 1)

In [115]:
asess_1["WoE"] = np.log(asess_1["%event"]/asess_1["%nonevent"])
asess_1["IV"] = asess_1["WoE"]*(asess_1["%event"] - asess_1["%nonevent"])


asess_1["IV"].sum() # Suspicious Predictor

5.072938766770576e-05

## 2. device_type columns

In [116]:
asess_2 = pd.crosstab(customers_trans1["device_type"], customers_trans1["label_churn"], margins=True, margins_name="Total")
asess_2

label_churn,0,1,Total
device_type,,,
Android,245312,189091,434403
iOS,74228,56910,131138
Total,319540,246001,565541


In [117]:
def nonevent(df) : 
    return (df[0]/319540)
def event(df) : 
    return (df[1]/246001)

    
asess_2["%nonevent"] = asess_2.apply(nonevent, axis = 1)
asess_2["%event"] = asess_2.apply(event, axis = 1)

In [118]:
asess_2["WoE"] = np.log(asess_2["%event"]/asess_2["%nonevent"])
asess_2["IV"] = asess_2["WoE"]*(asess_2["%event"] - asess_2["%nonevent"])


asess_2["IV"].sum() # Suspicious Predictor

5.131169128878947e-06

## 3. birthdate columns: generation, productive/nonproductive, remaja-tua

In [119]:
customers_trans3 = customers_trans1[['label_churn', 'birthdate']]

customers_trans3['birthdate'] = pd.to_datetime(customers_trans3['birthdate'])

#Last transaction in 2022, so we are assusme the current time is 2022
customers_trans3['age'] = 2022 -  customers_trans3['birthdate'].dt.year

customers_trans3['birthdate_year']= customers_trans3['birthdate'].dt.year

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [218]:
customers_trans3['age'].describe()

count    565541.000000
mean         26.445812
std           7.301839
min           6.000000
25%          21.000000
50%          26.000000
75%          31.000000
max          68.000000
Name: age, dtype: float64

In [120]:
def label_generation(x):
    if 15 <= x <= 64:
        return 'Productive'
    else:
        return 'Non-productives'

customers_trans3['label_generation'] = customers_trans3['age'].apply(label_generation)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [121]:
asess_3 = pd.crosstab(customers_trans3["label_generation"], customers_trans3["label_churn"], margins=True, margins_name="Total")
asess_3

label_churn,0,1,Total
label_generation,,,
Non-productives,9912,6943,16855
Productive,309628,239058,548686
Total,319540,246001,565541


In [122]:
def nonevent(df) : 
    return (df[0]/319540)
def event(df) : 
    return (df[1]/246001)

asess_3["%nonevent"] = asess_3.apply(nonevent, axis = 1)
asess_3["%event"] = asess_3.apply(event, axis = 1)

In [123]:
asess_3["WoE"] = np.log(asess_3["%event"]/asess_3["%nonevent"])
asess_3["IV"] = asess_3["WoE"]*(asess_3["%event"] - asess_3["%nonevent"])


asess_3["IV"].sum() # Weak Predictor

0.0002721941561807701

## 4. total_transaction

In [124]:
customers_trans4 = customers_trans1[['label_churn', 'total_transaction']]
customers_trans4

,label_churn,total_transaction
0,0,1
1,0,1
2,0,1
3,0,1
4,1,1
...,...,...
565536,1,2
565537,1,1
565538,1,1
565539,1,16


In [239]:
customers_trans4['total_transaction'].describe()

count    565541.000000
mean          2.123692
std           2.463846
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max         113.000000
Name: total_transaction, dtype: float64

In [125]:
def label_total_transaction(x):
    if x <= 2:
        return 'standard'
    else:
        return 'high'

customers_trans4['label_total_transaction'] = customers_trans4['total_transaction'].apply(label_total_transaction)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [126]:
asess_4 = pd.crosstab(customers_trans4["label_total_transaction"], customers_trans4["label_churn"], margins=True, margins_name="Total")
asess_4

label_churn,0,1,Total
label_total_transaction,,,
high,96010,25931,121941
standard,223530,220070,443600
Total,319540,246001,565541


In [127]:
def nonevent(df) : 
    return (df[0]/319540)
def event(df) : 
    return (df[1]/246001)

asess_4["%nonevent"] = asess_4.apply(nonevent, axis = 1)
asess_4["%event"] = asess_4.apply(event, axis = 1)

In [128]:
asess_4["WoE"] = np.log(asess_4["%event"]/asess_4["%nonevent"])
asess_4["IV"] = asess_4["WoE"]*(asess_4["%event"] - asess_4["%nonevent"])


asess_4["IV"].sum() # Medium Predictor

0.2522841638926472

## 5. quantity_item Column

In [129]:
customers_trans5 = customers_trans1[['label_churn', 'quantity_item']]
customers_trans5

,label_churn,quantity_item
0,0,5.0
1,0,1.0
2,0,1.0
3,0,1.0
4,1,1.0
...,...,...
565536,1,2.0
565537,1,1.0
565538,1,2.0
565539,1,22.0


In [130]:
customers_trans5['quantity_item'].describe()

count    565541.000000
mean          3.134006
std           4.321139
min           1.000000
25%           1.000000
50%           1.000000
75%           3.000000
max         180.000000
Name: quantity_item, dtype: float64

In [131]:
def label_quantity_item(x):
    if x == 1:
        return 'little'
    elif x <= 4:
        return 'standard'
    else:
        return 'high'

customers_trans5['label_quantity_item'] = customers_trans5['quantity_item'].apply(label_quantity_item)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == "__main__":


In [132]:
asess_5 = pd.crosstab(customers_trans5["label_quantity_item"], customers_trans5["label_churn"], margins=True, margins_name="Total")
asess_5

label_churn,0,1,Total
label_quantity_item,,,
high,76471,27494,103965
little,124213,163859,288072
standard,118856,54648,173504
Total,319540,246001,565541


In [133]:
def nonevent(df) : 
    return (df[0]/319540)
def event(df) : 
    return (df[1]/246001)

asess_5["%nonevent"] = asess_5.apply(nonevent, axis = 1)
asess_5["%event"] = asess_5.apply(event, axis = 1)

In [134]:
asess_5["WoE"] = np.log(asess_5["%event"]/asess_5["%nonevent"])
asess_5["IV"] = asess_5["WoE"]*(asess_5["%event"] - asess_5["%nonevent"])


asess_5["IV"].sum() #Strong Predictor

0.3237171642970971

## 6. total_amount Column

In [135]:
customers_trans6 = customers_trans1[['label_churn', 'total_amount']]
customers_trans6

,label_churn,total_amount
0,0,826597.0
1,0,163784.0
2,0,124932.0
3,0,247902.0
4,1,181237.0
...,...,...
565536,1,1068674.0
565537,1,366178.0
565538,1,485330.0
565539,1,67558976.0


In [136]:
customers_trans6['total_amount'].describe()

count    5.655410e+05
mean     2.512921e+06
std      1.146663e+07
min      1.520300e+04
25%      2.352600e+05
50%      4.054560e+05
75%      1.142561e+06
max      8.931705e+08
Name: total_amount, dtype: float64

In [137]:
def label_total_amount(x):
    if x <= 240000:
        return 'little'
    elif x <= 400000:
        return 'standard'
    else:
        return 'high'

customers_trans6['label_total_amount'] = customers_trans6['total_amount'].apply(label_total_amount)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == "__main__":


In [138]:
asess_6 = pd.crosstab(customers_trans6["label_total_amount"], customers_trans6["label_churn"], margins=True, margins_name="Total")
asess_6

label_churn,0,1,Total
label_total_amount,,,
high,192940,92686,285626
little,63855,83056,146911
standard,62745,70259,133004
Total,319540,246001,565541


In [139]:
def nonevent(df) : 
    return (df[0]/319540)
def event(df) : 
    return (df[1]/246001)

asess_6["%nonevent"] = asess_6.apply(nonevent, axis = 1)
asess_6["%event"] = asess_6.apply(event, axis = 1)

In [140]:
asess_6["WoE"] = np.log(asess_6["%event"]/asess_6["%nonevent"])
asess_6["IV"] = asess_6["WoE"]*(asess_6["%event"] - asess_6["%nonevent"])


asess_6["IV"].sum() # Medium Predictor

0.21277264595029943

## 7. use promo Column

In [141]:
customers_trans7 = customers_trans1[['label_churn', 'use_promo']]
customers_trans7

,label_churn,use_promo
0,0,1
1,0,0
2,0,1
3,0,1
4,1,1
...,...,...
565536,1,0
565537,1,1
565538,1,0
565539,1,16


In [142]:
def label_use_promo(x):
    if x <= 1:
        return 'barely'
    elif x <= 4:
        return 'regular'
    else:
        return 'frequent'

customers_trans7['label_use_promo'] = customers_trans7['use_promo'].apply(label_use_promo)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == "__main__":


In [143]:
asess_7 = pd.crosstab(customers_trans7["label_use_promo"], customers_trans7["label_churn"], margins=True, margins_name="Total")
asess_7

label_churn,0,1,Total
label_use_promo,,,
barely,262108,227445,489553
frequent,11119,4247,15366
regular,46313,14309,60622
Total,319540,246001,565541


In [144]:
def nonevent(df) : 
    return (df[0]/319540)
def event(df) : 
    return (df[1]/246001)

asess_7["%nonevent"] = asess_7.apply(nonevent, axis = 1)
asess_7["%event"] = asess_7.apply(event, axis = 1)

In [145]:
asess_7["WoE"] = np.log(asess_7["%event"]/asess_7["%nonevent"])
asess_7["IV"] = asess_7["WoE"]*(asess_7["%event"] - asess_7["%nonevent"])


asess_7["IV"].sum() # Medium Predictor

0.10399345517193605

## 8. freq_add_to_cart column

In [32]:
click_stream['event_name'].unique()

array(['HOMEPAGE', 'SCROLL', 'ADD_TO_CART', 'BOOKING', 'SEARCH',
       'ITEM_DETAIL', 'ADD_PROMO', 'CLICK', 'PROMO_PAGE'], dtype=object)

In [85]:
#
click_stream['event_time'] = pd.to_datetime(click_stream['event_time'])
click_stream['event_time'] = click_stream['event_time'].dt.strftime('%Y-%m')

#
click_stream_cart = click_stream[click_stream['event_name'] == 'ADD_TO_CART']
click_stream_cart.sort_values(['session_id', 'event_name'], inplace = True)

In [87]:
click_stream_cart1 = click_stream_cart.groupby(['session_id', 'event_time'], as_index= False).agg({'event_name':'count'})
click_stream_cart1.rename(columns = {'event_name':'add_to_cart_freq'}, inplace = True)
click_stream_cart1

,session_id,event_time,add_to_cart_freq
0,00001f29-3ede-4445-95f0-0178934e32ba,2019-05,1
1,00003eca-954b-4150-aee1-63fc62f395cf,2019-08,1
2,0000425b-ddc8-4459-a240-6ba814dac132,2019-03,1
3,00005746-17ec-449c-9c5c-699a3ea9304a,2022-03,1
4,0000649b-9514-4600-979b-42166a798996,2020-07,3
...,...,...,...
932666,ffffce53-1ba7-4d7d-a14a-ffa7079cfbff,2021-10,2
932667,ffffd3d9-79db-459e-a427-388204a7aac7,2021-11,4
932668,ffffdf8e-dea1-42bc-9839-7662675c0d38,2020-11,1
932669,fffff7cf-2a18-4303-9c12-aa836ae3b543,2021-02,5


In [67]:
click_stream_cart2 = click_stream_cart1.merge(customers_trans[['session_id', 'customer_id']], on = 'session_id', how = 'left')

#Let's ignore null value in customer_id columns because their customer do not have transaction at all
click_stream_cart2 = click_stream_cart2[~click_stream_cart2['customer_id'].isnull()]
click_stream_cart2

,session_id,event_time,add_to_cart_freq,customer_id
0,00001f29-3ede-4445-95f0-0178934e32ba,2019-05,1,74433.0
1,00003eca-954b-4150-aee1-63fc62f395cf,2019-08,1,43805.0
2,0000425b-ddc8-4459-a240-6ba814dac132,2019-03,1,90074.0
3,00005746-17ec-449c-9c5c-699a3ea9304a,2022-03,1,25255.0
4,0000649b-9514-4600-979b-42166a798996,2020-07,3,16868.0
...,...,...,...,...
1374201,ffffd3d9-79db-459e-a427-388204a7aac7,2021-11,4,15566.0
1374202,ffffd3d9-79db-459e-a427-388204a7aac7,2021-11,4,15566.0
1374203,ffffdf8e-dea1-42bc-9839-7662675c0d38,2020-11,1,65771.0
1374204,fffff7cf-2a18-4303-9c12-aa836ae3b543,2021-02,5,2353.0


In [97]:
click_stream_cart3 = click_stream_cart2.groupby(['customer_id', 'event_time'], as_index = False).agg({'add_to_cart_freq':'sum'})
click_stream_cart3.rename(columns = {'event_time':'created_at'}, inplace = True)
click_stream_cart3['created_at'] = pd.to_datetime(click_stream_cart4['created_at'])
click_stream_cart3

,customer_id,created_at,add_to_cart_freq
0,3.0,2018-08-01,2
1,3.0,2018-09-01,3
2,3.0,2018-10-01,3
3,3.0,2018-11-01,3
4,3.0,2018-12-01,4
...,...,...,...
606139,99998.0,2021-01-01,1
606140,99998.0,2021-07-01,3
606141,99998.0,2021-12-01,160
606142,99998.0,2022-01-01,128


In [101]:
customers_trans8 = customers_trans1[['customer_id', 'created_at', 'label_churn']].merge(click_stream_cart4, on = ['customer_id', 'created_at'], how = 'left')

#There customers have null value in add_to_cart_freq, I fill it with 0.
customers_trans8['add_to_cart_freq'].fillna(0, inplace = True)
customers_trans8

,customer_id,created_at,label_churn,add_to_cart_freq
0,3,2018-08-01,0,2.0
1,3,2018-09-01,0,3.0
2,3,2018-10-01,0,3.0
3,3,2018-11-01,0,3.0
4,3,2018-12-01,1,4.0
...,...,...,...,...
565536,99998,2020-07-01,1,0.0
565537,99998,2021-01-01,1,1.0
565538,99998,2021-07-01,1,3.0
565539,99998,2022-01-01,1,128.0


In [108]:
def label_add_to_cart_freq(x):
    if x <= 2:
        return 'barely'
    elif x <= 4:
        return 'standard'
    else:
        return 'frequent'

customers_trans8['label_add_to_cart_freq'] = customers_trans8['add_to_cart_freq'].apply(label_add_to_cart_freq)

In [109]:
asess_8 = pd.crosstab(customers_trans8["label_add_to_cart_freq"], customers_trans8["label_churn"], margins=True, margins_name="Total")
asess_8

label_churn,0,1,Total
label_add_to_cart_freq,,,
barely,173199,171129,344328
frequent,97576,39618,137194
standard,48765,35254,84019
Total,319540,246001,565541


In [110]:
def nonevent(df) : 
    return (df[0]/319540)
def event(df) : 
    return (df[1]/246001)

asess_8["%nonevent"] = asess_8.apply(nonevent, axis = 1)
asess_8["%event"] = asess_8.apply(event, axis = 1)

In [111]:
asess_8["WoE"] = np.log(asess_8["%event"]/asess_8["%nonevent"])
asess_8["IV"] = asess_8["WoE"]*(asess_8["%event"] - asess_8["%nonevent"])


asess_8["IV"].sum() # Medium Predictor

0.13124952693517206

## 9

In [19]:
customers_trans9 = customers_trans1[['label_churn' ,'created_at']]
customers_trans9

,label_churn,created_at
0,0,2018-08-01
1,0,2018-09-01
2,0,2018-10-01
3,0,2018-11-01
4,1,2018-12-01
...,...,...
565536,1,2020-07-01
565537,1,2021-01-01
565538,1,2021-07-01
565539,1,2022-01-01


In [20]:
customers_trans9.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 565541 entries, 0 to 565540
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   label_churn  565541 non-null  int64         
 1   created_at   565541 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1)
memory usage: 12.9 MB


In [24]:
customers_trans9['created_at'] = customers_trans9['created_at'].dt.strftime('%m')
customers_trans9

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,label_churn,created_at
0,0,08
1,0,09
2,0,10
3,0,11
4,1,12
...,...,...
565536,1,07
565537,1,01
565538,1,07
565539,1,01


In [32]:
customers_trans9.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 565541 entries, 0 to 565540
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   label_churn  565541 non-null  int64 
 1   created_at   565541 non-null  object
dtypes: int64(1), object(1)
memory usage: 12.9+ MB


In [34]:
customers_trans9['created_at'] = customers_trans9['created_at'].astype('int')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [35]:
customers_trans9[customers_trans9['created_at'] == 8]

,label_churn,created_at
0,0,8
11,0,8
23,0,8
35,0,8
75,1,8
...,...,...
565480,0,8
565490,1,8
565494,1,8
565507,0,8


In [45]:
def label_time_transaction(x):
    if x <= 3:
        return 'q1'
    elif x <= 6:
        return 'q2'
    elif x <= 9:
        return 'q3'
    else:
        return 'q4'

customers_trans9['label_time_transaction'] = customers_trans9['created_at'].apply(label_time_transaction)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [46]:
asess_9 = pd.crosstab(customers_trans9["label_time_transaction"], customers_trans9["label_churn"], margins=True, margins_name="Total")
asess_9

label_churn,0,1,Total
label_time_transaction,,,
q1,174156,125439,299595
q4,145384,120562,265946
Total,319540,246001,565541


In [47]:
def nonevent(df) : 
    return (df[0]/319540)
def event(df) : 
    return (df[1]/246001)

asess_9["%nonevent"] = asess_9.apply(nonevent, axis = 1)
asess_9["%event"] = asess_9.apply(event, axis = 1)

In [48]:
asess_9["WoE"] = np.log(asess_9["%event"]/asess_2["%nonevent"])
asess_9["IV"] = asess_9["WoE"]*(asess_9["%event"] - asess_2["%nonevent"])


asess_9["IV"].sum() # Useless for Predictor

0.004947388505506391

In [ ]:
#Template

def label_generation(x):
    if 15 <= x <= 64:
        return 'Productive'
    else:
        return 'Non-productives'

customers_trans3['label_generation'] = customers_trans3['age'].apply(label_generation)

asess_2 = pd.crosstab(customers_trans1["device_type"], customers_trans1["label_churn"], margins=True, margins_name="Total")
asess_2

def nonevent(df) : 
    return (df[0]/319540)
def event(df) : 
    return (df[1]/246001)

asess_3["%nonevent"] = asess_3.apply(nonevent, axis = 1)
asess_3["%event"] = asess_3.apply(event, axis = 1)

asess_2["WoE"] = np.log(asess_2["%event"]/asess_2["%nonevent"])
asess_2["IV"] = asess_2["WoE"]*(asess_2["%event"] - asess_2["%nonevent"])


asess_2["IV"].sum() # Suspicious for Predictor

## Conclusion:
